Calculate alt/az from RA/Dec/LST: 

$$ RA = LST - \arccos\left[{\sin(Alt) - \sin(Dec) \sin(Lat) \over \cos(Dec) \cos(Lat)}\right] $$
or
$$ LST = RA + \arccos\left[{\sin(Alt) - \sin(Dec) \sin(Lat) \over \cos(Dec) \cos(Lat)}\right] $$
References: see [here](http://en.wikibooks.org/wiki/General_Astronomy/Coordinate_Systems) and [here](http://en.wikipedia.org/wiki/Celestial_coordinate_system#Equatorial_.E2.86.90.E2.86.92_horizontal)

In [18]:
import numpy as np
from astropy.time import Time
import astropy.units as u

alt = 0.0
RA = 0
Dec = 0
lat = 0
latitude = 0*u.deg
longitude = 0*u.deg

def getHA(RA, Dec, lat, alt):
    return RA + np.arccos( (np.sin(alt) - np.sin(Dec)*np.sin(lat))
                          / (np.cos(Dec)*np.cos(lat)) )

def HA2UT(HA, time, longitude, kind='apparent'):
    '''
    time : instance of `astropy.time.Time`
    '''
    st = time.sidereal_time(kind, longitude)
    return st
    
HA = getHA(RA, Dec, lat, alt)
print("Hour angle: {0}".format(HA))
print("UT: {0}".format(HA2UT(HA, 
                       Time('2015-05-12 00:00:00', 
                            format='iso'), longitude)))


Hour angle: 1.57079632679


IndexError: (some) times are outside of range covered by IERS table.

In [25]:
print Time('2014-05-12 00:00:00',format='iso', 
           location=(longitude, latitude)).sidereal_time('apparent')

15h18m45.3241s
